In [1]:
from sympy import Mod, Integer
from sympy.core.numbers import mod_inverse

题目描述：分析题目附件中的代码，根据服务端所给出的密文，解密出所对应的明文消息  
题目附件中的代码：

In [2]:
# 模数
N_HEX = "FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFF7203DF6B21C6052B53BBF40939D54123" # 素数
MODULUS = Integer(int(N_HEX, 16)) # 将16进制字符串转换为整数，并创建一个sympy的Integer对象
MSG_PREFIX = "CryptoCup message:" # 消息前缀

In [3]:
# 加密函数
# 分组对称加密，使用自增的密钥
# 每个分组16字节，使用32字节的整数作为密钥
# 密钥自增1
# 使用模数进行加密和解密
def encrypt_message(message, key):
    # 添加前缀
    message_with_prefix = MSG_PREFIX + message
    message_bytes = message_with_prefix.encode('utf-8')
    message_len = len(message_bytes)
    num_blocks = (message_len + 15) // 16 # 分组数
    blocks = [message_bytes[i * 16:(i + 1) * 16] for i in range(num_blocks)] # 按16字节分组
    
    # 进行0填充
    # 如果最后一个分组不足 16 字节，则通过在末尾补零（0 填充）补齐到 16 字节
    blocks[-1] = blocks[-1].ljust(16, b'\x00')
    encrypted_blocks = []
    k = key

    # 加密每个分组
    # 
    for block in blocks:
        block_int = int.from_bytes(block, byteorder='big')
        encrypted_block_int = Mod(block_int * k, MODULUS)
        encrypted_blocks.append(encrypted_block_int)
        k += 1  # 密钥自增1
    
    # 将加密后的分组连接成最终的密文
    encrypted_message = b''.join(
        int(block_int).to_bytes(32, byteorder='big') for block_int in encrypted_blocks
    )
    return encrypted_message

# 解密函数
def decrypt_message(encrypted_message, key):
    num_blocks = len(encrypted_message) // 32
    blocks = [encrypted_message[i * 32:(i + 1) * 32] for i in range(num_blocks)]
    decrypted_blocks = []
    k = key

    # 解密每个分组
    for block in blocks:
        block_int = int.from_bytes(block, byteorder='big')
        key_inv = mod_inverse(k, MODULUS)
        decrypted_block_int = Mod(block_int * key_inv, MODULUS)
        decrypted_blocks.append(decrypted_block_int)
        k += 1  # 密钥自增1
    # 将解密后的分组连接成最终的明文
    decrypted_message = b''.join(
        int(block_int).to_bytes(16, byteorder='big') for block_int in decrypted_blocks
    )
    # 去除前缀
    if decrypted_message.startswith(MSG_PREFIX.encode('utf-8')):
        decrypted_message = decrypted_message[len(MSG_PREFIX):]

    return decrypted_message.rstrip(b'\x00').decode('utf-8')

# 测试
initial_key = Integer(0x123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0123456789ABCDEF0)
message = "Hello, this is a test message."
print("Original Message:", message)

# 加密
encrypted_message = encrypt_message(message, initial_key)
print("Encrypted Message (hex):", encrypted_message.hex())

# 解密
decrypted_message = decrypt_message(encrypted_message, initial_key)
print("Decrypted Message:", decrypted_message)

Original Message: Hello, this is a test message.
Encrypted Message (hex): 534ed400954f43256f50e9224595608726b3f016f8cece29ad868085526be54b1449d8eb3400703f429ae51b9675ce74aa8548240f176fe65b4ae4632f00eb157d852e4662abbb84a8a8914519beca68dbd6c138283a67d2b677c148ad396006
Decrypted Message: Hello, this is a test message.


模数和消息前缀已知，密钥未知   
设密文每个分组为c1, c2, ..., cn    
步骤一、首先将密文按32字节分组

In [ ]:
# 给定的密文（16进制字符串）
ciphertext_hex = encrypted_message.hex() ## 这里直接使用上面的加密信息
ciphertext_bytes = bytes.fromhex(ciphertext_hex)

# 提取第一个密文分组（32字节）
first_block = ciphertext_bytes[:32]
c1_int = int.from_bytes(first_block, 'big')

from sympy import isprime
isprime(MODULUS) # 检查模数是否为素数

True

设第一个分组加密前的明文为m1 = "CryptoCup mess"（16字节），第二个分组加密前的明文为m2 = "age:"后跟12个0（注意：第二个分组是16字节，其中前4字节是"age:"，后面12字节是0）   
已知：   
c1 = m1 * k (mod MODULUS)   
c2 = m2 * (k+1) (mod MODULUS)   
可以用第一个分组来求k：   
k = c1 * mod_inverse(m1, MODULUS) mod MODULUS   
步骤二、恢复初始密钥

In [5]:
# 计算第一个明文分组（前缀的前16字节）
m1_bytes = MSG_PREFIX.encode('utf-8')[:16]
m1_int = int.from_bytes(m1_bytes, 'big')

# 恢复初始密钥：k = c1 * m1^{-1} mod MODULUS
m1_inv = mod_inverse(m1_int, MODULUS)
initial_key = Mod(c1_int * m1_inv, MODULUS)

步骤三、从初始密钥中解密   
需要将三个步骤整合为一个函数   
下面实际调用题目的解密函数，因为密钥已知

In [ ]:
# 使用恢复的密钥解密整个密文
decrypted_msg = decrypt_message(ciphertext_bytes, initial_key)
print("Decrypted Message:", decrypted_msg)

Decrypted Message: Hello, this is a test message.


完整答案代码：  


```python
from sympy import Mod, Integer
from sympy.core.numbers import mod_inverse
from Crypto.Util.number import *

# 模数
N_HEX = "FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFF7203DF6B21C6052B53BBF40939D54123"
MODULUS = Integer(int(N_HEX, 16))
MSG_PREFIX = "CryptoCup message:"

# 解密函数
def decrypt_message(encrypted_message, key=-1):
    num_blocks = len(encrypted_message) // 32
    blocks = [encrypted_message[i * 32:(i + 1) * 32] for i in range(num_blocks)]

    decrypted_blocks = []

    k = key

    # 解密每个分组
    for block in blocks:
        block_int = int.from_bytes(block, byteorder='big')
        if k == -1:
            # get key
            decrypted_block_int = bytes_to_long(MSG_PREFIX[:16].encode())
            k = block_int * mod_inverse(decrypted_block_int, MODULUS) % MODULUS
            
        key_inv = mod_inverse(k, MODULUS)
        decrypted_block_int = Mod(block_int * key_inv, MODULUS)
        decrypted_blocks.append(decrypted_block_int)
        k += 1  # 密钥自增1

    # 将解密后的分组连接成最终的明文
    decrypted_message = b''.join(
        int(block_int).to_bytes(16, byteorder='big') for block_int in decrypted_blocks
    )

    # 去除前缀
    if decrypted_message.startswith(MSG_PREFIX.encode('utf-8')):
        decrypted_message = decrypted_message[len(MSG_PREFIX):]

    return decrypted_message.rstrip(b'\x00').decode('utf-8')


encrypted_message = bytes.fromhex('534ed400954f43256f50e9224595608726b3f016f8cece29ad868085526be54b1449d8eb3400703f429ae51b9675ce74aa8548240f176fe65b4ae4632f00eb157d852e4662abbb84a8a8914519beca68dbd6c138283a67d2b677c148ad396006')
decrypted_message = decrypt_message(encrypted_message)
print("Decrypted Message:", decrypted_message)
```